## This notebook is meant to accompany the trial_detection_Crab.py file that is included in this directory with additional comments. 

We will go through the code to produce Figure 3 of the associated BatAnalysis paper. 

First, we need to import the relevant packages. 

In [1]:
import glob
import os
import sys
import batanalysis as ba
import matplotlib.pyplot as plt
import numpy as np
from astropy.time import Time, TimeDelta
from astropy.io import fits
from pathlib import Path
import swiftbat

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


Now we need to set our data directory, which will contain all of our BAT survey data. 

In [2]:
newdir = Path("/Users/tparsota/Documents/CRAB_SURVEY_DATA")
ba.datadir(newdir, mkdir=True)

PosixPath('/Users/tparsota/Documents/CRAB_SURVEY_DATA')

Next, we query HEASARC for the observation IDs when the coordinates of the Crab was in the BAT FOV during the dates when the data for the 22 month survey was accumulated. We also ensure that the minimum amount of exposure that the Crab had to the BAT detector plane was $> 1000$ cm$^2$. 

In [4]:
object_name='Crab_Nebula_Pulsar'
queryargs = dict(time="2004-12-15 .. 2006-10-27", fields='All', resultmax=0)

#use swiftbat to create a bat source object
object_location = swiftbat.simbadlocation("Crab")
object_batsource = swiftbat.source(ra=object_location[0], dec=object_location[1], name=object_name)
table_everything = ba.from_heasarc(**queryargs)
minexposure = 1000     # cm^2 after cos adjust

#calculate the exposure with partial coding
exposures = np.array([object_batsource.exposure(ra=row['RA'], dec=row['DEC'], roll=row['ROLL_ANGLE'])[0] for row in table_everything])

#select the observations that have greater than the minimum desired exposure
table_exposed = table_everything[exposures > minexposure]

To download the data, we would then do:
```
result = ba.download_swiftdata(table_exposed)
obs_ids=[i for i in table_exposed['OBSID'] if result[i]['success']]
```

### OR

If we were continuing our analysis during some later point we would do:
```
obs_ids=[i.name for i in sorted(ba.datadir().glob("*")) if i.name.isnumeric()]
```

With the data downloaded and the observation IDs obtained, we can now analyze the BAT survey data. We specify that we want the BAT images to be cleaned from bright sources which have been detected at SNR=6. We also specify that all sources in the BatAnalysis or custom catalogs that have the column value ```ALWAYS_CLEAN==T``` should be cleaned as well.

After we define where the directory with all the pattern maps live in our system. Here, my patern map directory lies outside of where my BAT survey data have been downloaded and I want to include these pattern maps in my analyses for the computation of mosaic images later. Thus, I need to specify where these pattern noise maps live. If I do not then the pattern noise maps will not be included in the analyses and any later mosaic images that are created will suffer from this buildup of noise. 

In [ ]:
input_dict=dict(cleansnr=6,cleanexpr='ALWAYS_CLEAN==T')
noise_map_dir=Path("/Users/tparsota/Documents/PATTERN_MAPS/")
batsurvey_obs=ba.parallel.batsurvey_analysis(obs_ids, input_dict=input_dict, patt_noise_dir=noise_map_dir, nprocs=20)

Now, after all of the survey data has been processed, we can do a batch calculation of the pha files, the drm files, and subequently the spectral fitting for each observation and pointing ID. By default the spectrum that I am fitting to the BAT survey spectra is a ```cflux*po``` model from 14-195 keV. If the model parameters are not well constrained or if the Crab is not detected at a level of 3$\sigma$ above the background noise level then the function automatically tries to place 5$\sigma$ upper limits on the detection of the source. 

In the line below I set ```recalc=True``` which is useful for when the user wants to completely redo an operation using different input parameters. For example, if I want to run the below line using he defaults desribed above I can do so but if I want to change the model that is fitted, the level of detection necessary for automatically calculating upper limits or anything else, I simply set ```recalc=True``` and pass in the appropriate values and things will be updated within the ```batsurvey_obs``` objects appropriately. 

In [ ]:
batsurvey_obs=ba.parallel.batspectrum_analysis(batsurvey_obs, object_name, recalc=True,nprocs=14)

To get a quick glimpse of our results, we can use a convience function to plot the various values of interest for the Crab. 

The values that can be passed in are dicitonary values associated with the observation that can be accessed from the ```BatSurvey``` objects within the ```batsurvey_obs``` list. Some of these values are shown in the line below.

In [ ]:
fig, axes=ba.plot_survey_lc(batsurvey_obs, id_list=object_name, time_unit="UTC", values=["rate","snr", "flux", "PhoIndex", "exposure"])

### Next we want to do the mosaicing analysis

In order to do this step, we first have to group together all the BAT survey observations that we want to include in this step in our analysis. In most cases this is going to be all of our BAT survey observations. 

In [ ]:
outventory_file=ba.merge_outventory(batsurvey_obs)

If we wanted to continue our calculations from a point later on in our analysis, we can simply skip the above line and do:
```
outventory_file=Path("./path/to/outventory_all.fits")
```
since the above cell simply creates a fits file with all the BAt survey observations that we want included in the analysis and returns the full path to the file. 

Next, we need to define the time bins for which we will create mosaic images and analyze them. To test our code, we will use the same 1 month binning as the 22 month survey paper used. We specify the ```end_datetime``` explicitly but do not pass in a ```start_datetime``` value. This is because the ```start_datetime``` value is automatically set to be the first BAT survey observation rounded to the nearest whole ```timedelta``` value (ie the floor function applied to the earliest BAT survey date to the start of that month in this case).

In [ ]:
time_bins=ba.group_outventory(outventory_file, np.timedelta64(1, "M"), end_datetime=Time("2006-10-27"))

Now we can actually do the mosaicing calculation simply by doing. Where we will end up getting a list of mosaics for each month time bin and the total "time-integrated" mosaic.

In [ ]:
mosaic_list, total_mosaic=ba.parallel.batmosaic_analysis(batsurvey_obs, outventory_file, time_bins, nprocs=8)

To analyze the mosaic images we simply use the same call as we did for the BAT Survey data.

In [ ]:
mosaic_list=ba.parallel.batspectrum_analysis(mosaic_list, object_name, nprocs=11)
total_mosaic=ba.parallel.batspectrum_analysis(total_mosaic, object_name, nprocs=1)

And to plot our values of interest for each month, we would do:

In [ ]:
fig, axes=ba.plot_survey_lc(mosaic_list, id_list=object_name, time_unit="UTC", values=["rate","snr", "flux", "PhoIndex", "exposure"])

If we wanted to see the BAT survey data alongside the mosaic data, we would then do:

In [ ]:
fig, axes=ba.plot_survey_lc([batsurvey_obs,mosaic_list], id_list=object_name, time_unit="UTC", values=["rate","snr", "flux", "PhoIndex", "exposure"], same_figure=True)